In [ ]:
import pickle
import pandas as pd
from tensorflow.keras.models import load_model,Sequential
from tensorflow.keras.callbacks import TensorBoard,Callback,EarlyStopping
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset=pd.read_csv('/content/drive/MyDrive/ANN_Project/Churn_Modelling.csv')


In [128]:
dataset.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [127]:
dataset.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
dataset=dataset.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [ ]:
label_encoder_gender=LabelEncoder()
dataset['Gender']=label_encoder_gender.fit_transform(dataset['Gender'])

In [ ]:
one_hot_enccoder_geo=OneHotEncoder()
geo_encoded_array=one_hot_enccoder_geo.fit_transform(dataset[['Geography']])
geo_encoder_df=pd.DataFrame(geo_encoded_array.toarray(),columns=one_hot_enccoder_geo.get_feature_names_out(['Geography']))


In [ ]:
geo_encoder_df.head()

In [ ]:
# concate
dataset=pd.concat([dataset.drop('Geography',axis=1),geo_encoder_df],axis=1)


In [ ]:
dataset.head()

In [ ]:
from sklearn.model_selection import train_test_split
x=dataset.drop('EstimatedSalary',axis=1)
y=dataset['EstimatedSalary']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)



# apply standard scaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [126]:
print(x.columns)

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')


In [ ]:
#  now save label_encoder,one_hot_encoder and scaler as pkl file
pickle.dump(label_encoder_gender, open('/content/drive/MyDrive/ANN_Project/ANN_Regression_salary_Project/label_encoder_gender.pkl', 'wb'))
pickle.dump(one_hot_enccoder_geo, open('/content/drive/MyDrive/ANN_Project/ANN_Regression_salary_Project/one_hot_encoder_geo.pkl', 'wb'))

with open('/content/drive/MyDrive/ANN_Project/ANN_Regression_salary_Project/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), # HL1
    Dense(32,activation='relu'),  #HL2
    Dense(1)   #  output layer

])

In [ ]:
# 9)  compile the model  means assign or configure d/f perameter to model which we created
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [ ]:
#make folder
log_dir = "Regressionlogs/fit/static_run"
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
# 10)  Setup early stoping i.e if model not perform improvement then stop
early_stoping_callback=EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)


In [ ]:
# 11)  Train the model
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
      callbacks=[tensorflow_callback,early_stoping_callback]
)

In [ ]:
model.save('/content/drive/MyDrive/ANN_Project/ANN_Regression_salary_Project/Regression_model.h5')


In [ ]:
# 13) Load tensorboard for visual representation

%reload_ext tensorboard
%tensorboard --logdir Regressionlogs/fit/static_run


In [ ]:
test_loss,test_mae=model.evaluate(x_test,y_test)
print(test_mae)